In [1]:
# --- Imports & paths ---
import os
from pathlib import Path
import itertools
from datetime import datetime

import numpy as np
from obspy import read_inventory

from flovopy.asl.map import topo_map  # your function
from flovopy.core.mvo import dome_location

# --- Project inputs you already have somewhere else ---
PROJECTDIR = "/Users/GlennThompson/Dropbox/BRIEFCASE/SSADenver"
LOCALPROJECTDIR = "/Users/GlennThompson/work/PROJECTS/SSADenver_local"
OUTPUT_DIR = f"{LOCALPROJECTDIR}/asl_notebooks"

INVENTORY_XML = f"{PROJECTDIR}/MV.xml"
#DEM_TIF_FOR_BMAP = f"{PROJECTDIR}/channel_finder/02_dem_flipped_horizontal.tif"
DEM_TIF_FOR_BMAP = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean.tif")

# Montserrat default region (lon_min, lon_max, lat_min, lat_max)
REGION_DEFAULT = (-62.255, -62.135, 16.66, 16.84)

# Output folder
RUN_TAG = datetime.utcnow().strftime("topo_map_test_%Y%m%dT%H%M%S")
OUTDIR = Path(OUTPUT_DIR) / RUN_TAG
OUTDIR.mkdir(parents=True, exist_ok=True)

# Load inventory once (optional)
try:
    INV = read_inventory(INVENTORY_XML)
except Exception:
    INV = None

/var/folders/yj/6sjdgxm52t5d_p4w0lcrsrzr0000gp/T/ipykernel_84073/3953025037.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_TAG = datetime.utcnow().strftime("topo_map_test_%Y%m%dT%H%M%S")


In [7]:
# Example: shift 100 m north (in UTM)
DEM_SHIFTED = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean_shifted.tif")
DEM_SHIFTED = Path(PROJECTDIR) / "wgs84_s0.4_3_clean_shifted.tif"
from flovopy.dem.utils import shift_geotiff
shift_geotiff(DEM_TIF_FOR_BMAP, DEM_SHIFTED, dx=0.0, dy=-16.0)

In [3]:
#dem = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit/wgs84_s0.4_3.tif")
#dem = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean.tif")


fig = topo_map(
    show=False,
    region=REGION_DEFAULT,
    dem_tif=str(DEM_SHIFTED),          # <- your GeoTIFF in WGS84
    inv=INV,                   # <- plot stations from inventory
    add_labels=True,           # <- draw station codes next to symbols
    topo_color=True,           # <- colored topography (uses 'geo' CPT by default)
    #level_interval=100,      # <- topo contours (meters)
    hillshade_azimuth=135,
    hillshade_altitude=30,
    add_shading=True,
    figsize=6.0,               # inches; projection defaults to f"M{figsize}i"
    title="Montserrat DEM + Stations",
    frame=True,                # axes and ticks
    outfile=str(DEM_SHIFTED).replace('.tif', "_topo_map.png")
)

/Users/glennthompson/Developer/flovopy/flovopy/asl/map.py:148: RasterioDeprecationWarning: statistics() will be removed in 2.0.0. Please switch to stats().
  stats = src.statistics(1) if hasattr(src, "statistics") else None
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
coast [ERROR]: Pen name None not recognized!
coast [WARNING]: Representation of pen width (None) not recognized. Using default.
/opt/anaconda3/envs/flovopy_plus/lib/python3.12/site-packages/obspy/core/inventory/network.py:321: UserWarning: Found more than one matching channel metadata. Returning first.
  warnings.warn(msg)


topo_map: Saving to /Users/GlennThompson/work/PROJECTS/SSADenver_local/wgs84_s0.4_3_clean_shifted_topo_map.png


In [4]:
import itertools
import numpy as np

def run_case(
    *,
    region,
    zoom_level,
    add_labels,
    topo_color,
    cmap,
    use_dem_tif: bool,
    title: str = None,
    figsize: float = 6.0,
    hillshade_azimuth: float = 135,
    hillshade_altitude: float = 30,
    add_shading: bool = True,          # NEW
):
    # Build filename that encodes params (now includes shading)
    parts = [
        f"reg_{'explicit' if region else 'auto'}",
        f"zoom_{zoom_level}",
        f"labels_{'on' if add_labels else 'off'}",
        f"topocol_{'on' if topo_color else 'off'}",
        f"cmap_{cmap if cmap is not None else 'none'}",
        f"dem_{'tif' if use_dem_tif else 'earthrelief'}",
        f"shading_{'on' if add_shading else 'off'}",
    ]
    fname = "__".join(parts) + ".png"
    outpng = OUTDIR / fname

    fig, used_region = topo_map(
        show=False,
        return_region=True,      # so we can assert region behavior
        zoom_level=zoom_level,
        inv=INV,
        add_labels=add_labels,
        topo_color=topo_color,
        cmap=cmap,
        region=region,
        dem_tif=(DEM_TIF_FOR_BMAP if use_dem_tif else None),
        resolution="01s",
        figsize=figsize,
        hillshade_azimuth=hillshade_azimuth,
        hillshade_altitude=hillshade_altitude,
        add_shading=add_shading,  # NEW
        frame=True,
        title=title,
        dome_location=dome_location,     # optional: show dome marker
    )

    # Region sanity: if explicit region was provided, check topo_map honored it
    if region is not None:
        exp = np.array(region, dtype=float)
        got = np.array(used_region, dtype=float)
        if not np.allclose(exp, got, rtol=0, atol=1e-6):
            print(f"[WARN] Region mismatch! expected={exp}, got={got}, file={outpng.name}")

    fig.savefig(str(outpng), dpi=300)
    print(f"[OK] Saved {outpng.name} | used_region={used_region}")
    return outpng

In [ ]:
# --- parameter grids ---
regions = [
    REGION_DEFAULT,  # explicit
    None,            # let zoom rules decide
]
zoom_levels = [0, 1, 2]
label_opts = [False, True]
topo_color_opts = [False, True]
cmaps = [None, "topo", "gray", "land"]
dem_opts = [False, True]      # False => EarthRelief, True => GeoTIFF
shading_opts = [False, True]  # NEW

MAX_OUTPUTS = 60
count = 0

for region, zoom_level, add_labels, topo_color, cmap, use_dem_tif, add_shading in itertools.product(
    regions, zoom_levels, label_opts, topo_color_opts, cmaps, dem_opts, shading_opts
):
    # Skip conflicting combos (optional):
    if (not topo_color) and (cmap == "topo"):
        continue
    # If using two-color "land", shading usually looks odd — feel free to skip:
    # if cmap == "land" and add_shading:
    #     continue

    run_case(
        region=region,
        zoom_level=zoom_level,
        add_labels=add_labels,
        topo_color=topo_color,
        cmap=cmap,
        use_dem_tif=use_dem_tif,
        title="topo_map test",
        hillshade_azimuth=135,
        hillshade_altitude=30,
        add_shading=add_shading,  # NEW
    )

    count += 1
    if count >= MAX_OUTPUTS:
        print(f"[INFO] Reached MAX_OUTPUTS={MAX_OUTPUTS}")
        break

print(f"[DONE] Wrote images to: {OUTDIR}")

# A few targeted cases:
run_case(region=None, zoom_level=0, add_labels=True, topo_color=True,
         cmap="topo", use_dem_tif=False, title="Auto region (ER)", add_shading=True)

run_case(region=REGION_DEFAULT, zoom_level=0, add_labels=False, topo_color=False,
         cmap="land", use_dem_tif=True, title="Explicit region (GeoTIFF)", add_shading=False)

run_case(region=None, zoom_level=2, add_labels=True, topo_color=True,
         cmap="topo", use_dem_tif=False, title="Zoom 2 (ER, topo)", add_shading=True)

run_case(region=None, zoom_level=2, add_labels=True, topo_color=False,
         cmap="gray", use_dem_tif=False, title="Zoom 2 (ER, gray)", add_shading=True)